<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Weather-dataset-analysis" data-toc-modified-id="Weather-dataset-analysis-1">Weather dataset analysis</a></span><ul class="toc-item"><li><span><a href="#Observation-types" data-toc-modified-id="Observation-types-1.1">Observation types</a></span></li><li><span><a href="#Hourly-weather-observations" data-toc-modified-id="Hourly-weather-observations-1.2">Hourly weather observations</a></span></li><li><span><a href="#Every-6-hours-weather-observations" data-toc-modified-id="Every-6-hours-weather-observations-1.3">Every 6 hours weather observations</a></span></li><li><span><a href="#Daily-weather-observations" data-toc-modified-id="Daily-weather-observations-1.4">Daily weather observations</a></span></li><li><span><a href="#Minutely-weather-observations" data-toc-modified-id="Minutely-weather-observations-1.5">Minutely weather observations</a></span></li></ul></li><li><span><a href="#Solar-declination-angle" data-toc-modified-id="Solar-declination-angle-2">Solar declination angle</a></span></li><li><span><a href="#Daily-aggregation" data-toc-modified-id="Daily-aggregation-3">Daily aggregation</a></span></li><li><span><a href="#Hourly-aggregations" data-toc-modified-id="Hourly-aggregations-4">Hourly aggregations</a></span></li></ul></div>

# Data Preporation

In [3]:
#Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm
import math

* Read UR data and replace missing values

In [19]:
names = [
    'Date', 'Electric_KWAC_Power', 'Energy_(KWh)', 'Inverter_1', 'Inverter_2',
    'Inverter_3', 'Inverter_4', 'Inverter_5', 'Inverter_6', 'Inverter_7',
    'Inverter_8', 'Solar_Irradiance', 'Power_for_Tesla_Battery',
    'Current_Charge_Level(KWh)'
]

df = pd.read_excel('Data/URData_update.xlsx',sheet_name='Data',skiprows=4,names=names)

df.columns

missing_values = ['I/O Timeout', 'No Result', 'No Data', 'Pt Created',0]

word_replace = {'I/O Timeout':0, 'No Result':0, 'No Data':0, 'Pt Created':0}

df = df[~df['Electric_KWAC_Power'].isin(missing_values)]

df.replace(
    {
        'Inverter_1': word_replace,
        'Inverter_2': word_replace,
        'Inverter_3': word_replace,
        'Inverter_4': word_replace,
        'Inverter_5': word_replace,
        'Inverter_6': word_replace,
        'Inverter_7': word_replace,
        'Inverter_8': word_replace,
        'Solar_Irradiance': word_replace
    },
    inplace=True)
df.drop(index=[0,1],inplace= True)

## Weather dataset analysis

* Read and explore weather dataset

In [4]:
df_weather = pd.read_csv('Data/ROC.csv')

# Date to datetime
df_weather.DATE = pd.to_datetime(df_weather.DATE, format='%Y-%m-%dT%H:%M:%S')
df_weather['REPORT_TYPE'].value_counts()

/Users/marinakupina/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,30,31,42,43,48,53,58,59,69,71,76,88,95) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


FM-15    7839
FM-16    1947
FM-12    1296
SHEF      457
SOD       324
SOM        10
SY-MT       1
Name: REPORT_TYPE, dtype: int64

### Observation types
There are different sources of weather data. The main difference is observation frequency.

* FM-15 - every hour (definition: Aviation routine reports	FM 15 (METAR))
* FM-16 - (definition: Special aviation weather reports	FM 16 (SPECI))
* FM-12 - every 6 hours (definition: Intermediate synoptic hour	FM 12 (SYNOP)/FM 13 (SHIP))
* SOD - every day; contains information about sunrise and sunset

In [5]:
pd.set_option('display.max_columns', None)
df_weather[df_weather['REPORT_TYPE']=='FM-15'].head().T

,0,2,4,5,6
STATION,72529014768,72529014768,72529014768,72529014768,72529014768
DATE,2020-01-01 00:54:00,2020-01-01 01:54:00,2020-01-01 02:54:00,2020-01-01 03:54:00,2020-01-01 04:54:00
REPORT_TYPE,FM-15,FM-15,FM-15,FM-15,FM-15
SOURCE,7,7,7,7,7
AWND,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
ShortDurationPrecipitationValue180,NaN,NaN,NaN,NaN,NaN
Sunrise,NaN,NaN,NaN,NaN,NaN
Sunset,NaN,NaN,NaN,NaN,NaN
TStorms,NaN,NaN,NaN,NaN,NaN


### Hourly weather observations

In [6]:
df_weather_hourly = df_weather[df_weather['REPORT_TYPE']=='FM-15'].dropna(axis=1,
                                           thresh=len(df_weather[df_weather['REPORT_TYPE']=='FM-15'])*0.7)

In [7]:
df_weather_hourly.shape

(7839, 23)

In [8]:
df_weather_hourly['SkyConditions_level'] = df_weather_hourly['HourlySkyConditions'].astype(str).apply(lambda x: 
                                                                                        x.split(':')[-1][0:2])

In [9]:
df_weather_hourly.columns = ['Station', 'Date', 'Report_type', 'Source', 'BackupElements',
       'BackupEquipment', 'BackupName', 'HourlyAltimeterSetting',
       'HourlyDewPointTemperature', 'HourlyDryBulbTemperature',
       'HourlyPrecipitation', 'HourlyRelativeHumidity',
       'HourlySeaLevelPressure', 'HourlySkyConditions',
       'HourlyStationPressure', 'HourlyVisibility', 'HourlyWetBulbTemperature',
       'HourlyWindDirection', 'HourlyWindSpeed', 'REM', 'REPORT_TYPE.1',
       'SOURCE.1', 'WindEquipmentChangeDate', 'SkyConditions_level']
df_weather_hourly.columns

Index(['Station', 'Date', 'Report_type', 'Source', 'BackupElements',
       'BackupEquipment', 'BackupName', 'HourlyAltimeterSetting',
       'HourlyDewPointTemperature', 'HourlyDryBulbTemperature',
       'HourlyPrecipitation', 'HourlyRelativeHumidity',
       'HourlySeaLevelPressure', 'HourlySkyConditions',
       'HourlyStationPressure', 'HourlyVisibility', 'HourlyWetBulbTemperature',
       'HourlyWindDirection', 'HourlyWindSpeed', 'REM', 'REPORT_TYPE.1',
       'SOURCE.1', 'WindEquipmentChangeDate', 'SkyConditions_level'],
      dtype='object')

### Every 6 hours weather observations

In [10]:
df_weather_every6hours = df_weather[df_weather['REPORT_TYPE']=='FM-12'].dropna(axis=1,
                                           thresh=len(df_weather[df_weather['REPORT_TYPE']=='FM-12'])*0.7)

In [11]:
df_weather_every6hours.shape

(1296, 22)

### Daily weather observations

In [12]:
df_weather_daily = df_weather[df_weather['REPORT_TYPE']=='SOD  '].dropna(axis=1,
                                           thresh=len(df_weather[df_weather['REPORT_TYPE']=='SOD  '])*0.7)

In [13]:
df_weather_daily.shape

(324, 32)

In [14]:
time_sunrise = pd.to_datetime(df_weather_daily['Sunrise'], format='%H%M')
time_sunset = pd.to_datetime(df_weather_daily['Sunset'], format='%H%M')
df_weather_daily['Day_duration'] = time_sunset - time_sunrise

### Minutely weather observations

In [27]:
df.sort_values(by='Date',inplace=True)

In [28]:
df_final_minutely = pd.merge_asof(df, df_weather_hourly, on='Date',direction="nearest")

In [29]:
df_final_minutely.shape

(421757, 37)

In [30]:
df['Electric_KWAC_Power'] = pd.to_numeric(df['Electric_KWAC_Power'])
df['Date_day_hour'] = df['Date'].apply(lambda x: x.strftime("%Y-%m-%d %H:00:00"))
df_hour_agg = df.groupby('Date_day_hour').mean().reset_index()
df_hour_agg['Date_day_hour'] = pd.to_datetime(df_hour_agg['Date_day_hour'], unit="ns")
df_final_hourly = pd.merge_asof(df_hour_agg, df_weather_hourly,left_on = 'Date_day_hour',
                                right_on='Date',direction="nearest")

In [31]:
df_final_hourly.dtypes
df_final_hourly.shape

(7031, 35)

In [32]:
df_final_hourly.fillna(0,inplace=True)

## Solar declination angle

In [33]:
solar_angles = pd.read_csv('Data/Daily_Solar_Information.csv', index_col=0)

jan1 = pd.to_datetime(solar_angles.Date, format='%m/%d/%y')[1]
solar_angles['Days_Jan1'] = (
    pd.to_datetime(solar_angles.Date, format='%m/%d/%y') -
    jan1).astype('timedelta64[D]')
solar_angles['sun_declination_angle'] = solar_angles['Days_Jan1'].apply(
    lambda x: 0.39795 * math.cos(0.01720249 * (x - 173)))

# Date to datetime
solar_angles.Date = pd.to_datetime(solar_angles.Date)
solar_angles.head()
solar_angles.describe()
solar_angles['Hours.After.Midnight'].value_counts()
solar_angles.drop(['Hours.After.Midnight', 'Sunrise.Time', 'Sunset.Time'],
                  axis=1,
                  inplace=True)

## Daily aggregation 

In [35]:
df_weather_daily['Date_day'] = pd.to_datetime(
    df_weather_daily['DATE'].apply(lambda x: x.strftime("%Y-%m-%d")))

In [36]:
df['Electric_KWAC_Power'] = pd.to_numeric(df['Electric_KWAC_Power'])
df['Date_day'] = pd.to_datetime(df['Date'].apply(lambda x: x.strftime("%Y-%m-%d")))
df_day_agg = df.groupby('Date_day').mean().reset_index()
df_final_daily = pd.merge_asof(df_day_agg,
                               df_weather_daily,
                               on='Date_day',
                               direction="nearest")

In [37]:
#Day duration
time_sunrise = pd.to_datetime(df_final_daily['Sunrise'], format='%H%M')
time_sunset = pd.to_datetime(df_final_daily['Sunset'], format='%H%M')
df_final_daily['Day_duration'] = (time_sunset -
                                  time_sunrise).astype('timedelta64[m]')

In [38]:
df_final_daily['DailyPeakWindDirection'] = df_final_daily[
                            'DailyPeakWindDirection'].astype(str).apply(
                            lambda x: x[:-1] if 's' in x else x).astype(float)

In [39]:
df_final_daily['DailyPeakWindSpeed'] = df_final_daily['DailyPeakWindSpeed'].astype(str).apply(lambda x: 
                                                                x[:-1] if 's' in x else x).astype(float)

In [40]:
df_final_daily['DailyPrecipitation'] = df_final_daily['DailyPrecipitation'].replace('T',0).astype(float)

In [42]:
df_final_daily['DailySnowDepth'].replace('T',1,inplace=True)
df_final_daily['DailySnowDepth'].fillna(0,inplace=True)
df_final_daily['DailySnowDepth'] = df_final_daily['DailySnowDepth'].astype(int)

In [43]:
df_final_daily['DailySnowfall'].replace('T',0.5,inplace=True)
df_final_daily['DailySnowfall'].fillna(0,inplace=True)
df_final_daily['DailySnowfall'] = df_final_daily['DailySnowfall'].astype(float)

In [44]:
df_final_hourly.drop([
    'Station', 'REM', 'REPORT_TYPE.1', 'SOURCE.1', 'HourlySkyConditions',
    'BackupName', 'BackupElements', 'Report_type', 'Source', 'BackupEquipment',
    'WindEquipmentChangeDate'], axis=1, inplace=True)

In [45]:
df_final_hourly['Date_ymd'] = df_final_hourly['Date_day_hour'].apply(lambda x: x.strftime("%Y-%m-%d"))

In [52]:
df_final_hourly.drop(index= 5987,axis=0,inplace=True)

In [53]:
# To type float/int
df_final_hourly.HourlyWetBulbTemperature = df_final_hourly.HourlyWetBulbTemperature.astype(
                            str).apply(lambda x: x[:-1] if '*' in x else x).astype(float)
df_final_hourly.HourlyDewPointTemperature = df_final_hourly.HourlyDewPointTemperature.astype(
                            str).apply(lambda x: x[:-1] if 's' in x else x).astype(float)
df_final_hourly.HourlySeaLevelPressure = df_final_hourly.HourlySeaLevelPressure.astype(
                            str).apply(lambda x: x[:-1] if 's' in x else x).astype(float)
df_final_hourly.HourlyStationPressure = df_final_hourly.HourlyStationPressure.astype(
                            str).apply(lambda x: x[:-1] if 's' in x else x).astype(float)
df_final_hourly.HourlyVisibility = df_final_hourly.HourlyVisibility.astype(
                            str).apply(lambda x: x[:-1] if 'V' in x else x).astype(float)
df_final_hourly.HourlyAltimeterSetting = df_final_hourly.HourlyAltimeterSetting.astype(
                            str).apply(lambda x: x[:-1] if 's' in x else x).astype(float)
df_final_hourly.HourlyPrecipitation = df_final_hourly.HourlyPrecipitation.replace('T',
                            0).astype(str).apply(lambda x: x[:-1] if 's' in x else x).astype(float)
df_final_hourly.HourlyWindDirection = df_final_hourly.HourlyWindDirection.replace(
                            'VRB', '000').astype(int)
df_final_hourly.SkyConditions_level = df_final_hourly.SkyConditions_level.replace(
                            'na', 0).astype(int)

In [54]:
df_final_hourly.HourlyDryBulbTemperature = df_final_hourly.HourlyDryBulbTemperature.astype(int)

In [56]:
df_final_hourly.HourlyDewPointTemperature = df_final_hourly.HourlyDewPointTemperature.astype(int)
df_final_hourly.HourlyDryBulbTemperature = df_final_hourly.HourlyDryBulbTemperature.astype(int)
df_final_hourly.HourlyRelativeHumidity = df_final_hourly.HourlyRelativeHumidity.astype(int)
df_final_hourly.HourlyWetBulbTemperature = df_final_hourly.HourlyWetBulbTemperature.astype(int)

In [58]:
aggs = ['min', 'max', 'mean', 'median', 'std']
result = df_final_hourly.groupby('Date_ymd').agg({'HourlyDryBulbTemperature':aggs,
                                                  'HourlyRelativeHumidity':aggs,
                                                  'HourlyDewPointTemperature':aggs,
                                                  'Electric_KWAC_Power':aggs+['sum'], 
                                                  'Inverter_1':aggs+['sum'], 'Inverter_2':aggs+['sum'],
                                                  'Inverter_3':aggs+['sum'], 'Inverter_4':aggs+['sum'], 
                                                  'Inverter_5':aggs+['sum'], 'Inverter_6':aggs+['sum'], 
                                                  'Inverter_7':aggs+['sum'], 'Inverter_8':aggs+['sum'], 
                                                  'Solar_Irradiance':aggs+['sum'],
                                                  'HourlyWetBulbTemperature':aggs, 
                                                  'HourlyWindSpeed':aggs,
                                                 }).reset_index()

In [59]:
result.columns = ['_'.join(col).strip() for col in result.columns.values]

In [60]:
result.rename(columns={"Date_ymd_": "Date"},inplace=True)
result['Date'] = pd.to_datetime(result['Date'])

In [61]:
df_final_daily.rename(columns={"Date_day": "Date"},inplace=True)

In [62]:
df_agg_daily = pd.merge(pd.merge(df_final_daily,result,on='Date'),solar_angles,on='Date')

In [63]:
df_agg_daily.columns[:60]

Index(['Date', 'Electric_KWAC_Power', 'Inverter_1', 'Inverter_2', 'Inverter_3',
       'Inverter_4', 'Inverter_5', 'Inverter_6', 'Inverter_7', 'Inverter_8',
       'Solar_Irradiance', 'STATION', 'DATE', 'REPORT_TYPE', 'SOURCE',
       'BackupElements', 'BackupEquipment', 'BackupName',
       'DailyAverageDewPointTemperature', 'DailyAverageDryBulbTemperature',
       'DailyAverageRelativeHumidity', 'DailyAverageSeaLevelPressure',
       'DailyAverageStationPressure', 'DailyAverageWetBulbTemperature',
       'DailyAverageWindSpeed', 'DailyCoolingDegreeDays',
       'DailyDepartureFromNormalAverageTemperature', 'DailyHeatingDegreeDays',
       'DailyMaximumDryBulbTemperature', 'DailyMinimumDryBulbTemperature',
       'DailyPeakWindDirection', 'DailyPeakWindSpeed', 'DailyPrecipitation',
       'DailySnowDepth', 'DailySnowfall', 'DailySustainedWindDirection',
       'DailySustainedWindSpeed', 'REM', 'REPORT_TYPE.1', 'SOURCE.1',
       'Sunrise', 'Sunset', 'WindEquipmentChangeDate', 'Day_dur

In [64]:
df_agg_daily['Temperature_max-min'] = df_agg_daily[
                        'HourlyDryBulbTemperature_max'] - df_agg_daily[
                        'HourlyDryBulbTemperature_min']

mean_power_June21 = df_agg_daily[df_agg_daily['Date'] == '2020-06-21']['Electric_KWAC_Power_mean']
df_agg_daily['Power_mean_deviation_June21'] = df_agg_daily['Electric_KWAC_Power_mean'] - mean_power_June21

sum_power_June21 = df_agg_daily[df_agg_daily['Date'] == '2020-06-21']['Electric_KWAC_Power_sum']
df_agg_daily['Power_sum_deviation_June21'] = df_agg_daily['Electric_KWAC_Power_sum'] - sum_power_June21

mean_solar_irradiance_June21 = df_agg_daily[df_agg_daily['Date'] == '2020-06-21']['Solar_Irradiance_mean']
df_agg_daily['Irradiance_deviation_June21'] = df_agg_daily[ 'Solar_Irradiance_mean'] - mean_solar_irradiance_June21

sum_solar_irradiance_June21 = df_agg_daily[df_agg_daily['Date'] == '2020-06-21']['Solar_Irradiance_sum']
df_agg_daily['Irradiance_sum_deviation_June21'] = df_agg_daily['Solar_Irradiance_sum'] - sum_solar_irradiance_June21

In [ ]:
#Save to csv file
df_agg_daily.to_csv('All_aggregations_daily.csv')

## Hourly aggregations

In [65]:
df_final_minutely.dtypes

Date                         datetime64[ns]
Electric_KWAC_Power                  object
Energy_(KWh)                         object
Inverter_1                          float64
Inverter_2                          float64
Inverter_3                          float64
Inverter_4                          float64
Inverter_5                          float64
Inverter_6                          float64
Inverter_7                          float64
Inverter_8                          float64
Solar_Irradiance                    float64
Power_for_Tesla_Battery              object
Current_Charge_Level(KWh)            object
Station                               int64
Report_type                          object
Source                               object
BackupElements                       object
BackupEquipment                      object
BackupName                           object
HourlyAltimeterSetting               object
HourlyDewPointTemperature            object
HourlyDryBulbTemperature        

In [66]:
df_final_minutely.fillna(0,inplace=True)

In [71]:
df_final_minutely.drop(index=df_final_minutely[df_final_minutely.HourlyDewPointTemperature=='*'].index,axis=0,inplace=True)

In [73]:
# To type float/int
df_final_minutely.HourlyDewPointTemperature = df_final_minutely.HourlyDewPointTemperature.astype(str).apply(lambda x: 
                                                                x[:-1] if 's' in x else x).astype(float)
df_final_minutely.HourlySeaLevelPressure = df_final_minutely.HourlySeaLevelPressure.astype(str).apply(lambda x: 
                                                                x[:-1] if 's' in x else x).astype(float)
df_final_minutely.HourlyStationPressure = df_final_minutely.HourlyStationPressure.astype(str).apply(lambda x: 
                                                                x[:-1] if 's' in x else x).astype(float)
df_final_minutely.HourlyVisibility = df_final_minutely.HourlyVisibility.astype(str).apply(lambda x: 
                                                                x[:-1] if 'V' in x else x).astype(float)
df_final_minutely.HourlyAltimeterSetting = df_final_minutely.HourlyAltimeterSetting.astype(str).apply(lambda x: 
                                                                x[:-1] if 's' in x else x).astype(float)
df_final_minutely.HourlyPrecipitation = df_final_minutely.HourlyPrecipitation.replace('T',0).astype(str).apply(lambda x: 
                                                                x[:-1] if 's' in x else x).astype(float)
df_final_minutely.HourlyWindDirection = df_final_minutely.HourlyWindDirection.replace('VRB','000').astype(int)
df_final_minutely.SkyConditions_level = df_final_minutely.SkyConditions_level.replace('na',0).astype(int)

In [74]:
df_final_minutely.HourlyDryBulbTemperature = df_final_minutely.HourlyDryBulbTemperature.astype(float)

In [76]:
df_final_minutely.drop(index=df_final_minutely[df_final_minutely.HourlyRelativeHumidity =='*'].index,axis=0,inplace = True)

In [77]:
df_final_minutely.HourlyRelativeHumidity = df_final_minutely.HourlyRelativeHumidity.astype(float)
df_final_minutely.HourlyWetBulbTemperature = df_final_minutely.HourlyWetBulbTemperature.astype(float)
df_final_minutely.HourlyDewPointTemperature = df_final_minutely.HourlyDewPointTemperature.astype(float)          
df_final_minutely.HourlyDryBulbTemperature = df_final_minutely.HourlyDryBulbTemperature.astype(float)

In [78]:
aggs = ['min', 'max', 'mean', 'median', 'std']
df_final_minutely['Date_ymdh'] = df_final_minutely['Date'].apply(lambda x: x.strftime("%Y-%m-%d %H"))
result = df_final_minutely.groupby('Date_ymdh').agg({'HourlyDryBulbTemperature':aggs,
                                                  'HourlyRelativeHumidity':aggs,
                                                  'HourlyDewPointTemperature':aggs,
                                                  'Electric_KWAC_Power':aggs+['sum'],
                                                  'Inverter_1':aggs+['sum'], 'Inverter_2':aggs+['sum'],
                                                  'Inverter_3':aggs+['sum'], 'Inverter_4':aggs+['sum'], 
                                                  'Inverter_5':aggs+['sum'], 'Inverter_6':aggs+['sum'], 
                                                  'Inverter_7':aggs+['sum'], 'Inverter_8':aggs+['sum'], 
                                                  'Solar_Irradiance':aggs+['sum'], 
                                                  'HourlyWetBulbTemperature':aggs, 'HourlyWindDirection':aggs, 
                                                  'HourlyWindSpeed':aggs,'SkyConditions_level':aggs}).reset_index()

In [79]:
result.columns = ['_'.join(col).strip() for col in result.columns.values]

In [80]:
result.head()

,Date_ymdh_,HourlyDryBulbTemperature_min,HourlyDryBulbTemperature_max,HourlyDryBulbTemperature_mean,HourlyDryBulbTemperature_median,HourlyDryBulbTemperature_std,HourlyRelativeHumidity_min,HourlyRelativeHumidity_max,HourlyRelativeHumidity_mean,HourlyRelativeHumidity_median,HourlyRelativeHumidity_std,HourlyDewPointTemperature_min,HourlyDewPointTemperature_max,HourlyDewPointTemperature_mean,HourlyDewPointTemperature_median,HourlyDewPointTemperature_std,Electric_KWAC_Power_min,Electric_KWAC_Power_max,Electric_KWAC_Power_mean,Electric_KWAC_Power_median,Electric_KWAC_Power_std,Electric_KWAC_Power_sum,Inverter_1_min,Inverter_1_max,Inverter_1_mean,Inverter_1_median,Inverter_1_std,Inverter_1_sum,Inverter_2_min,Inverter_2_max,Inverter_2_mean,Inverter_2_median,Inverter_2_std,Inverter_2_sum,Inverter_3_min,Inverter_3_max,Inverter_3_mean,Inverter_3_median,Inverter_3_std,Inverter_3_sum,Inverter_4_min,Inverter_4_max,Inverter_4_mean,Inverter_4_median,Inverter_4_std,Inverter_4_sum,Inverter_5_min,Inverter_5_max,Inverter_5_mean,Inverter_5_median,Inverter_5_std,Inverter_5_sum,Inverter_6_min,Inverter_6_max,Inverter_6_mean,Inverter_6_median,Inverter_6_std,Inverter_6_sum,Inverter_7_min,Inverter_7_max,Inverter_7_mean,Inverter_7_median,Inverter_7_std,Inverter_7_sum,Inverter_8_min,Inverter_8_max,Inverter_8_mean,Inverter_8_median,Inverter_8_std,Inverter_8_sum,Solar_Irradiance_min,Solar_Irradiance_max,Solar_Irradiance_mean,Solar_Irradiance_median,Solar_Irradiance_std,Solar_Irradiance_sum,HourlyWetBulbTemperature_min,HourlyWetBulbTemperature_max,HourlyWetBulbTemperature_mean,HourlyWetBulbTemperature_median,HourlyWetBulbTemperature_std,HourlyWindDirection_min,HourlyWindDirection_max,HourlyWindDirection_mean,HourlyWindDirection_median,HourlyWindDirection_std,HourlyWindSpeed_min,HourlyWindSpeed_max,HourlyWindSpeed_mean,HourlyWindSpeed_median,HourlyWindSpeed_std,SkyConditions_level_min,SkyConditions_level_max,SkyConditions_level_mean,SkyConditions_level_median,SkyConditions_level_std
0,2020-02-01 00,28.0,28.0,28.000000,28.0,0.000000,88.0,88.0,88.00,88.0,0.000000,25.0,25.0,25.0,25.0,0.0,-0.026958,0.011129,-0.007914,-0.007914,0.011274,-0.474856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,27.0,27.000000,27.0,0.000000,0,0,0.0,0,0.000000,0.0,0.0,0.000000,0.0,0.000000,8,8,8.0,8,0.0
1,2020-02-01 01,28.0,29.0,28.583333,29.0,0.497167,85.0,88.0,86.25,85.0,1.491501,25.0,25.0,25.0,25.0,0.0,0.011775,0.097115,0.054111,0.054099,0.025446,3.246659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,28.0,27.583333,28.0,0.497167,0,0,0.0,0,0.000000,0.0,0.0,0.000000,0.0,0.000000,8,8,8.0,8,0.0
2,2020-02-01 02,28.0,29.0,28.416667,28.0,0.497167,85.0,88.0,86.75,88.0,1.491501,25.0,25.0,25.0,25.0,0.0,0.098574,0.184606,0.141590,0.141590,0.025466,8.495380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,28.0,27.416667,27.0,0.497167,0,0,0.0,0,0.000000,0.0,0.0,0.000000,0.0,0.000000,8,8,8.0,8,0.0
3,2020-02-01 03,28.0,28.0,28.000000,28.0,0.000000,88.0,88.0,88.00,88.0,0.000000,25.0,25.0,25.0,25.0,0.0,0.186064,0.272096,0.229080,0.229080,0.025466,13.744793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,27.0,27.000000,27.0,0.000000,0,240,140.0,240,119.320108,0.0,5.0,2.916667,5.0,2.485836,8,8,8.0,8,0.0
4,2020-02-01 04,28.0,28.0,28.000000,28.0,0.000000,88.0,88.0,88.00,88.0,0.000000,25.0,25.0,25.0,25.0,0.0,0.273554,0.359

In [81]:
result.rename(columns={"Date_ymdh_": "Date"},inplace=True)
result['Date'] = pd.to_datetime(result['Date'])

In [82]:
df_final_hourly['Date'] = df_final_hourly['Date_day_hour'].apply(lambda x: x.strftime("%Y-%m-%d %H"))
df_final_hourly['Date'] = pd.to_datetime(df_final_hourly['Date'],format="%Y-%m-%d %H")

In [83]:
df_final_hourly.drop(['Date_day_hour','Date_ymd'],axis=1, inplace=True)

In [86]:
df_agg_hourly = pd.merge(df_final_hourly,result,on='Date')
df_agg_hourly['Date_ymd'] = pd.to_datetime(df_final_hourly['Date'].apply(lambda x: x.strftime("%Y-%m-%d")))
df_agg_hourly = pd.merge(df_agg_hourly,solar_angles,left_on='Date_ymd',right_on='Date')

In [92]:
df_agg_hourly['Temperature_max-min'] = df_agg_hourly[
    'HourlyDryBulbTemperature_max'] - df_agg_hourly[
        'HourlyDryBulbTemperature_min']

mean_power_June21 = df_agg_hourly[df_agg_hourly['Date_x'] == '2020-06-21']['Electric_KWAC_Power']
df_agg_hourly['Power_mean_deviation_June21'] = df_agg_hourly[
    'Electric_KWAC_Power'] - mean_power_June21

sum_power_June21 = df_agg_hourly[df_agg_hourly['Date_x'] == '2020-06-21']['Electric_KWAC_Power']
df_agg_hourly['Power_sum_deviation_June21'] = df_agg_hourly[
    'Electric_KWAC_Power'] - sum_power_June21

mean_solar_irradiance_June21 = df_agg_hourly[
    df_agg_hourly['Date_x'] == '2020-06-21']['Solar_Irradiance_mean']
df_agg_hourly['Irradiance_deviation_June21'] = df_agg_hourly[
    'Solar_Irradiance_mean'] - mean_solar_irradiance_June21

sum_solar_irradiance_June21 = df_agg_hourly[
    df_agg_hourly['Date_x'] == '2020-06-21']['Solar_Irradiance_sum']
df_agg_hourly['Irradiance_sum_deviation_June21'] = df_agg_hourly[
    'Solar_Irradiance_sum'] - sum_solar_irradiance_June21

In [93]:
df_agg_hourly.shape

(7027, 134)

In [ ]:
# Save to cvs file
df_agg_hourly.to_csv('All_aggregations_hourly_updates.csv')